In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
import googlemaps
import requests
import time


In [48]:
# Import datasets 

# HSI (Hazardous Site Inventory) contains a list of contaminated sites in Georgia that need to be cleaned up
# Use to identify contaminated sites in Georgia (includes landfills, superfund)
df_hsi = pd.read_excel("../../data/raw/scoring_indicators/July-2024-Hazardous-Site-Inventory.xlsx")

# TRI (Toxic Release Inventory) contains how much toxic chemicals are released into the environment
# Use to identify sites in Georgia that release toxic chemicals
df_tri = pd.read_csv("../../data/raw/scoring_indicators/waste_hazardous_chemicals.csv")

# UST (Underground Storage Tanks) contains information on underground storage tanks in Georgia
df_ust = pd.read_csv("../../data/raw/scoring_indicators/Facilities_GA.csv")

# Food Access Research Atlas contains information on food access in Georgia
# Use to identify food deserts in Georgia
df_food_deserts = pd.read_csv("../../data/raw/scoring_indicators/food_access_research_atlas.csv")

df_cdr = pd.read_csv("../../data/processed/scoring_indicators/cdr_industrial_manufacturing_facilities.csv")

df_frs = pd.read_csv("../../data/processed/scoring_indicators/frs_facilities_naics_sic.csv")

df_rcra = pd.read_csv("../../data/processed/scoring_indicators/rcra_facilities.csv")

/var/folders/w2/jx7kgvv14cx4km6tl2rymsmw0000gn/T/ipykernel_39243/3492858725.py:20: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_frs = pd.read_csv("../../data/processed/scoring_indicators/frs_facilities_naics_sic.csv")


In [3]:
# Import datasets 
rural_tracts_housing = gpd.read_file("../../data/raw/shapefiles/USDA_Rural_Housing_by_Tract_7054655361891465054/USDA_Rural_Housing_by_Tract.shp")
rural_ga_tracts_housing = rural_tracts_housing.to_crs(epsg=4326)

# Census tracts for mapping development to rural or metro areas 
census_tracts = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp")
ga_tracts = census_tracts.to_crs(epsg=4326)

In [5]:
test_couty = gpd.read_file("../../data/raw/shapefiles/tl_2024_us_county/tl_2024_us_county.shp")

In [6]:
test_couty

,STATEFP,COUNTYFP,COUNTYNS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,0500000US31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477563042,10772508,+41.9158651,-096.7885168,"POLYGON ((-96.55525 41.82892, -96.55524 41.827..."
1,53,069,01513275,53069,0500000US53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680980773,61564428,+46.2946377,-123.4244583,"POLYGON ((-123.72755 46.2645, -123.72756 46.26..."
2,35,011,00933054,35011,0500000US35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016818941,29090018,+34.3592729,-104.3686961,"POLYGON ((-104.89337 34.08894, -104.89337 34.0..."
3,31,109,00835876,31109,0500000US31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169269508,22850511,+40.7835474,-096.6886584,"POLYGON ((-96.68493 40.5233, -96.69219 40.5231..."
4,31,129,00835886,31129,0500000US31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645201,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.2737 40.1184, -98.27374 40.1224,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3230,13,123,00351260,13123,0500000US13123,Gilmer,Gilmer County,06,H1,G4020,None,None,None,A,1103804462,12337139,+34.6905232,-084.4548113,"POLYGON ((-84.30237 34.57832, -84.30329 34.577..."
3231,27,135,00659513,27135,0500000US27135,Roseau,Roseau County,06,H1,G4020,None,None,None,A,4329782927,16924046,+48.7610683,-095.8215042,"POLYGON ((-95.25857 48.88666, -95.25707 48.885..."
3232,28,089,00695768,28089,0500000US28089,Madison,Madison County,06,H1,G4020,298,27140,None,A,1849796735,72079469,+32.6343703,-090.0341603,"POLYGON ((-90.14883 32.40026, -90.1489 32.4001..."
3233,48,227,01383899,48227,0500000US48227,Howard,Howard County,06,H1,G4020,None,13700,None,A,2333034781,8846149,+32.3034298,-101.4387208,"POLYGON ((-101.18138 32.21252, -101.18138 32.2..."


In [ ]:
GOOGLE_API_KEY = ""

In [3]:
gmaps_client = googlemaps.Client(key=GOOGLE_API_KEY)


### Helper Functions

In [4]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Great-circle distance (miles) between two points on Earth.
    Used for the 0.25 mi radius checks (undesirable).
    """
    R = 3959.87433  # Earth's radius in miles
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) ** 2
         + math.cos(math.radians(lat1))
         * math.cos(math.radians(lat2))
         * math.sin(dlon / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [5]:
def determine_pool(lat, lon):
    return "Atlanta Metro Pool"
    ## placeholder for determining if development is in a rural or metro pool

In [19]:
def get_route_distance_miles(lat1, lon1, lat2, lon2, mode="driving"):
    """
    Uses Google Distance Matrix to get route distance in miles.
    mode can be "driving" or "walking".
    """
    start_time = time.time()

    resp = gmaps_client.distance_matrix(
        origins=[(lat1, lon1)],
        destinations=[(lat2, lon2)],
        mode=mode
    )
    # Parse out distance (meters) from resp
    element = resp["rows"][0]["elements"][0]
    # print("get_route_distance_miles took ", time.time() - start_time, "seconds")
    if element["status"] == "OK":
        dist_m = element["distance"]["value"]  # meters
        return dist_m / 1609.34
    else:
        return None

In [7]:
DESIRABLE_AMENITIES = {
    "national_big_box_store": {"group": 1, "google_type": ["department_store"], "name_contains": ["Walmart", "Target", "Costco", "BJ's", "Sam's Club"]}, 
    "retail_store": {"group": 2, "google_type": ["clothing_store", "home_goods_store"]},
    "grocery_store": {"group": 1, "google_type": ["grocery_or_supermarket", 'supermarket'], "type_not_contains": ['convience_store']}, 
    "restaurant": {"group": 2, "google_type": ["retaurant"]},
    "hospital": {"group": 1, "google_type": ["hospital"], "name_not_contains": ["Outpatient"]}, 
    "medical_clinic": {"group": 1, "google_type": ["doctor"], "name_contains": ["urgent care", "medical clinic", "immediate care", "physicians", "dentist"]}, 
    "pharmacy": {"group": 1, "google_type": ["pharmacy"]},
    # "licensed_childcare": {"group": 1},
    "technical_college": {"group": 2, "google_type":['university']}, 
    "school": {"group": 1, "google_type": ["primary_school", "secondary_school", "school"]},
    "town_square": {"group": 1, "google_type": ['city_hall', 'courthouse'] },
    "community_center": {"group": 1, "google_type": ["community_center", 'gym', 'pool']},
    "public_park": {"group": 1, "google_type": ["park"]},
    # "large_public_park": {"group": 1, "google_type": ["park"]}, maybe turn this into just a park 
    # "small_public_park": {"group": 2, "google_type": ["park"]},
    "library": {"group": 1, "google_type": ["library"]},
    "fire_police_station": {"group": 2, "google_type": ["fire_station", "police"]},
    "bank": {"group": 2, "google_type": ["bank"]}, 
    "place_of_worship": {"group": 2, "google_type": ["place_of_worship"]},
    "post_office": {"group": 2, "google_type": ["post_office"]},
}

In [8]:
def get_desirable_amenities_nearby(site_lat, site_lon, place_type, radius_meters = 10000):

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius_meters,
        "key": GOOGLE_API_KEY,
        "type": place_type
    }
    resp = requests.get(base_url, params=params).json()
    if "results" in resp:
        return resp["results"]
    return []

In [18]:
def google_places_search(lat, lon, place_type, radius_meters=10000):
    """
    Query the Google Places API (Nearby Search) for a specific type,
    within a certain radius in meters. Returns a list of place dicts.
    """
    start_time = time.time()

    # The official doc: https://developers.google.com/maps/documentation/places/web-service/search
    # We might do something like:
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius_meters,
        "key": GOOGLE_API_KEY,
        "type": place_type
    }
    resp = requests.get(base_url, params=params).json()
    # print("google_places_search took ", time.time() - start_time, "seconds")
    if "results" in resp:
        return resp["results"]
    return []

NOTE: DONT CALL THE GOOGLE API FOR EVERY LONG AND LATITUDE POINT PUT IN 
- PULL ALL DESIRABLE ACTIVITIES FOR THE ENTIRETY OF GEORGIA
- Do an exhaustive coverage of georgia (do it essentially as a grid pull)
- Don't want to have API called for every development site 


Need to decide how to store the data (for desirable)
- have csv with type, name, lat, long

Have function to read in the csv and then have fetch nearest location function (returns type, name, lat, long) 
- e.g., someone wants to know where the closest grocery store is 


In [21]:
def fetch_nearest_desirable(lat, lon, amenity_key, config, pool_type):
    """
    Finds the nearest location for a single amenity category.
    Returns a distance in miles (route distance) if found, else None.
    """
    start_time = time.time()

    google_types = config.get("google_type")
    if not google_types:
        return None


    min_dist = None
    
    for gtype in google_types:
        places = google_places_search(lat, lon, gtype, radius_meters=10000)  
        
        for place in places:
            place_name = place["name"].lower()
            place_types = place.get("types", [])


            if any(bad_sub.lower() in place_name for bad_sub in config.get("name_not_contains", [])):
                continue

            req_subs = config.get("name_contains", [])
            if req_subs and not any(req.lower() in place_name for req in req_subs):
                continue
            
            not_contains = config.get("type_not_contains", [])

            if any(sub.lower() in place_name for sub in not_contains):
                continue
            

            type_not_contains_list = config.get("type_not_contains", [])
            if any(tnc.lower() in place_types for tnc in type_not_contains_list):
                continue

            # # If there's a "store_names" list, we must match exactly or partially
            # store_names = config.get("store_names", [])
            # if store_names:
            #     # For big box store, we only accept if name has e.g. "walmart" or "target"
            #     # We'll do partial match. 
            #     # If none match, skip
            #     if not any(sname.lower() in name for sname in store_names):
            #         continue
            
            # # If there's a "contains" list, we accept only if name has one of these
            # if "contains" in config:
            #     c_list = config["contains"]
            #     # If none of them appear in 'name', skip
            #     if not any(x.lower() in name for x in c_list):
            #         continue
            
            # Special case: check bank vs. ATM only
            # If google_type=bank but the name is "ATM" only, skip
            # We'll rely on "not_contains": ["atm"] above, but that is partial.
            # If the place is literally "ABC ATM", it's excluded.
            
            # If it's a park, we might attempt to check area
            # if gtype == "park":
            #     # Attempt to get area
            #     # place_id = p["place_id"]
            #     # area_sqft = get_park_area_in_sqft(place_id)  # if known
            #     # Then decide if it's large or small
            #     pass
            dest_lat = place["geometry"]["location"]["lat"]
            dest_lon = place["geometry"]["location"]["lng"]

            dist_miles = get_route_distance_miles(lat, lon, dest_lat, dest_lon, mode="driving")
            if dist_miles is not None:
                if min_dist is None or dist_miles < min_dist:
                    min_dist = dist_miles
                    print(f"Found {amenity_key}: {place.get('name')} at {place.get('vicinity')} — {round(dist_miles, 2)} miles")

    # print("fetch_nearest_desirable took ", time.time() - start_time, "seconds")
    return min_dist

In [11]:
def compute_desirable_points(pool_type, group_num, distance_miles):
    """
    Applies the QAP table for Group 1 or Group 2, 
    according to distance and pool type (Metro or Rural).
    """
    if distance_miles is None:
        return 0.0
    
    # Metro includes 'Atlanta Metro Pool' and 'Other Metro Pool'
    if pool_type in ("Atlanta Metro Pool", "Other Metro Pool"):
        if group_num == 1:
            if distance_miles <= 0.5:
                return 2.5
            elif distance_miles <= 1.0:
                return 2.0
            elif distance_miles <= 1.5:
                return 1.5
            else:
                return 0.0
        else:  # group 2
            if distance_miles <= 0.5:
                return 2.0
            elif distance_miles <= 1.0:
                return 1.5
            elif distance_miles <= 1.5:
                return 1.0
            else:
                return 0.0
    else:
        # Rural
        if group_num == 1:
            if distance_miles <= 0.5:
                return 2.5
            elif distance_miles <= 1.0:
                return 2.0
            elif distance_miles <= 2.5:
                return 1.5
            else:
                return 0.0
        else:
            if distance_miles <= 0.5:
                return 2.0
            elif distance_miles <= 1.0:
                return 1.5
            elif distance_miles <= 2.5:
                return 1.0
            else:
                return 0.0

In [17]:
def calculate_desirable_score(lat, lon, pool_type):
    """
    For each amenity in DESIRABLE_AMENITIES:
      - find nearest instance
      - compute points
    Sum points, cap at 20.
    Return that sum.
    """
    start_time = time.time()

    total_pts = 0.0
    for amenity_key, config in DESIRABLE_AMENITIES.items():
        group_num = config["group"]
        dist = fetch_nearest_desirable(lat, lon, amenity_key, config, pool_type)
        pts = compute_desirable_points(pool_type, group_num, dist)
        total_pts += pts
    # Cap at 20 points 
    # print("calculate_desirable_score took ", time.time() - start_time, "seconds")

    return min(total_pts, 20)

In [23]:
lat = 33.823971
lon = -84.616553  # Downtown Atlanta

pool_type = determine_pool(lat, lon)  # returns 'Atlanta Metro Pool'

score = calculate_desirable_score(lat, lon, pool_type)

print("Final Desirable Score:", score)

Found national_big_box_store: Target at 4125 Austell Road, Austell — 2.49 miles
Found retail_store: Village Boutique at United States — 1.63 miles
Found retail_store: Kay-La's Boutique at 2141 Kingston Court Suite 111, Marietta — 0.6 miles
Found grocery_store: Food Depot at 1355 East-West Connector A19, Austell — 3.12 miles
Found grocery_store: Supermercado Super Mi Pueblo de Austell at 411 Maxham Road #600, Austell — 2.0 miles
Found grocery_store: Jerez Mexican Market at Clay Plaza Shopping Center, 4968 Austell Road #126, Austell — 1.07 miles
Found restaurant: Atlanta at Atlanta — 17.61 miles
Found restaurant: Hampton Inn & Suites ATL-Six Flags at 999 Bob Arnold Boulevard, Lithia Springs — 4.53 miles
Found restaurant: Holiday Inn Express Hotel Atlanta NW - Powder Springs at 3741 Tramore Pointe Parkway SW, Austell — 4.12 miles
Found restaurant: Village Boutique at United States — 1.63 miles
Found hospital: AICA Orthopedics at 530 Thornton Road #210, Lithia Springs — 2.45 miles
Found ho

### Undesirable Activities

In [ ]:
UNDESIRABLE_ACTIVITIES = {
    "auto_repair_station": {"google_type": "car_repair"},
    "commercial_livestock": {"google_type": "farm"},
    "excessive_light": {"google_type": ["casino", "stadium", 'night_club']},
    "excessive_noise": {'google_type': ['airport']}, 
    "laundry_facility": {"google_type": "laundry"},
    "gas_station": {"google_type": "gas_station"},
}

In [ ]:
def google_places_undesirables(lat, lon):
    """
    Returns how many distinct undesirable 'types' from Google Places 
    exist within 0.25 mi (as the crow flies).
    NOTE: The QAP says 0.25 mi "radius," so we do a Haversine filter 
    after retrieving up to ~400 meters search radius.
    """
    # Small radius in meters (0.25 miles ~ 400m)
    RADIUS_M = 400
    found_types = []
    
    for cat, cfg in UNDESIRABLE_ACTIVITIES.items():
        gtypes = cfg["google_type"]
        if isinstance(gtypes, str):
            gtypes = [gtypes]
        
        for gtype in gtypes:
            places = google_places_search(lat, lon, gtype, radius_meters=RADIUS_M)
            # For each place, do a precise Haversine check
            for p in places:
                p_lat = p["geometry"]["location"]["lat"]
                p_lon = p["geometry"]["location"]["lng"]
                dist = haversine_distance(lat, lon, p_lat, p_lon)
                if dist <= 0.25:
                    found_types.append(cat)
    return set(found_types)

In [ ]:
UNDESIRABLE_CATEGORIES = [
    "junkyard",
    "dump",
    "landfill",
    "materials_storage",
    "commercial_livestock",
    "odor_producing",       
    "chemical_activities",
    "heavy_manufacturing",
    "industrial_development",
    "hazardous_inventory", 
    "gas_station_ust_leak",
    "dry_cleaner_contamination"
]

Create undesirable dataset
- type, site name, lat, long 

In [ ]:
def identify_undesirable_activities(site_lat, site_lon, radius_miles=0.25):
    """
    Given a LIHTC site lat/lon, identify which of the QAP "undesirable activities" 
    are present within 0.25 miles (or user-specified radius).

    Returns: A set of category strings (e.g., {"junkyard", "landfill"}).
    """
    # Create a Point geometry for the site location
    site_point = Point(site_lon, site_lat)

    # Convert 0.25 miles to a buffer in degrees 
    deg_per_mile = 1.0 / 69.0
    buffer_deg = radius_miles * deg_per_mile
    search_area = site_point.buffer(buffer_deg)

    # Undesirables stored here
    found_undesirables = set()

    # ------------------------------------------------------------------------
    # RCRA (Resource Conservation and Recovery Act) dataset 
    # ------------------------------------------------------------------------
    try:
        # Convert to GeoDataFrame
        rcra_gdf = gpd.GeoDataFrame(
            df_rcra,
            geometry=[Point(xy) for xy in zip(df_rcra["LONGITUDE83"], df_rcra["LATITUDE83"])],
            crs="EPSG:4326"
        )
        # Filter those within 0.25 miles via bounding box or direct intersection (naive intersection)
        rcra_in_area = rcra_gdf[rcra_gdf.geometry.within(search_area)]

        # Identifying landfill, materials storage, incinerators, treatment facilities
        for idx, row in rcra_in_area.iterrows():
            op_tsdf = str(row.get("OPERATING_TSDF", "")).upper()
            naics = str(row.get("NAICS_CODE", ""))

            if "L" in op_tsdf:
                found_undesirables.add("landfill")

            if "S" in op_tsdf:
                found_undesirables.add("materials_storage")

            if "I" in op_tsdf:
                found_undesirables.add("Incinerator")

            if "T" in op_tsdf:
                found_undesirables.add("Treatment")

            if "H" in op_tsdf:
                found_undesirables.add("wate_management")

            # NAICs
            try:
                if left(naics, 2) == "31" or left(naics, 2) == "32" or left(naics, 2) == "33":
                    found_undesirables.add("heavy_manufacturing and/or chemical activities")
            except:
                pass
    except Exception as e:
        print("Could not process RCRA dataset:", e)

    # ------------------------------------------------------------------------
    # CDR (Chemical Data Reporting) dataset
    # ------------------------------------------------------------------------
    try:
        cdr_gdf = gpd.GeoDataFrame(
            df_cdr,
            geometry=[Point(xy) for xy in zip(df_cdr["SITE LONGITUDE"], df_cdr["SITE LATITUDE"])],
            crs="EPSG:4326"
        )
        cdr_in_area = cdr_gdf[cdr_gdf.geometry.within(search_area)]

        for idx, row in cdr_in_area.iterrows():
            naics = str(row.get("SITE NAICS CODE 1", ""))
            try:
                if left(naics, 2) == "31" or left(naics, 2) == "32" or left(naics, 2) == "33":
                    found_undesirables.add("heavy_manufacturing and/or chemical activities")
            except:
                pass

    except Exception as e:
        print("Could not process CDR dataset:", e)

    # ------------------------------------------------------------------------
    # FRS (Facility Registry Service) dataset 
    # ------------------------------------------------------------------------
    try:
        frs_gdf = gpd.GeoDataFrame(
            df_frs,
            geometry=[Point(xy) for xy in zip(df_frs["LONGITUDE_MEASURE"], df_frs["LATITUDE_MEASURE"])],
            crs="EPSG:4326"
        )
        frs_in_area = frs_gdf[frs_gdf.geometry.within(search_area)]

        for idx, row in frs_in_area.iterrows():
            naics = str(row.get("NAICS_CODE", ""))
            try:
                if left(naics, 2) == "31" or left(naics, 2) == "32" or left(naics, 2) == "33":
                    found_undesirables.add("heavy_manufacturing and/or chemical activities")
                if left(naics, 3) == '112':
                    found_undesirables.add("commercial_livestock")
            except:
                pass

    except Exception as e:
        print("Could not process FRS dataset:", e)

    # ------------------------------------------------------------------------
    # TRI (Toxic Release Inventory) dataset
    # ------------------------------------------------------------------------
    try:
        tri_gdf = gpd.GeoDataFrame(
            df_tri,
            geometry=[Point(xy) for xy in zip(df_tri["Longitude"], df_tri["Latitude"])],
            crs="EPSG:4326"
        )
        tri_in_area = tri_gdf[tri_gdf.geometry.within(search_area)]

        for idx, row in tri_in_area.iterrows():
            found_undesirables.add("chemical_activities")
            # maybe "industrial_development"
            found_undesirables.add("industrial_development")
    except Exception as e:
        print("Could not process TRI dataset:", e)

    # ------------------------------------------------------------------------
    # HSI (Hazardous Site Inventory) dataset
    # ------------------------------------------------------------------------
    try:
        hsi_gdf = gpd.GeoDataFrame(
            df_hsi,
            geometry=[Point(xy) for xy in zip(df_hsi["Longitude"], df_hsi["Lattitude"])],
            crs="EPSG:4326"
        )
        hsi_in_area = hsi_gdf[hsi_gdf.geometry.within(search_area)]

        if not hsi_in_area.empty:
            found_undesirables.add("hazardous_inventory")


        for idx, row in hsi_in_area.iterrows():
            # Possibly parse "Site Name" to see if it's "landfill", "junkyard", "dump" 
            site_name = str(row.get("Site Name", "")).lower()
            if "landfill" in site_name:
                found_undesirables.add("landfill")
            if "junkyard" in site_name:
                found_undesirables.add("junkyard")
            if "dump" in site_name:
                found_undesirables.add("dump")
            # If it's known "dry cleaner" with contamination?
            if "dry clean" in site_name:
                found_undesirables.add("dry_cleaner_contamination")
            # If it's a known "gas station" with contamination?
            if "gas station" in site_name or "fuel station" in site_name:
                found_undesirables.add("gas_station_ust_leak")

    except Exception as e:
        print("Could not process HSI dataset:", e)
    return found_undesirables


In [ ]:
def check_hazard_datasets(lat, lon):
    """
    Check other data sources (RCRA, TRI, Hazardous Sites Inventory, etc.)
    for sites within 0.25 miles.
    
    Return a set of 'undesirable' categories matched, or empty set if none.
    """
    # We'll do a geospatial approach: 
    #   - read each dataset as a GeoDataFrame
    #   - buffer the site point by 0.25 miles
    #   - do an intersection to find any sites in that buffer
    site_point = Point(lon, lat)
    buffer_geom = site_point.buffer(0.25 * 1609.34)  # ~0.25 miles in meters is ~402.335m
                                                    # but shapely is in degrees if not projected.
                                                    # Typically, you'd reproject to a local CRS or handle carefully.
    undesirables_found = set()
    
    # Example with a Hazardous Sites shapefile
    # We assume the shapefile is in a projected coordinate system in feet or meters.
    try:
        haz_gdf = gpd.read_file(HAZARDOUS_SITES_SHP)
        # Ensure same CRS if needed:
        # if haz_gdf.crs != "EPSG:XXXX":
        #     haz_gdf = haz_gdf.to_crs("EPSG:XXXX")
        
        # Filter with bounding box or direct intersection
        intersects_mask = haz_gdf.geometry.intersects(buffer_geom)
        if intersects_mask.any():
            # We found some hazard sites within 0.25 miles
            # The QAP lumps them as an undesirable if it’s e.g. a contamination site
            undesirables_found.add("hazardous_site")
    except:
        pass
    
    # Similarly for RCRA, TRI, etc. 
    # You can unify them under categories like "chemical_mfg", 
    # "industrial_disposal", "contaminated_site", etc., 
    # each mapped to a 2-point deduction.
    
    return undesirables_found

In [ ]:
def compute_undesirable_deduction(lat, lon):
    """
    Each undesirable activity => -2 points
    Summation of all distinct categories found near the site.
    """
    google_undes = google_places_undesirables(lat, lon)
    hazard_undes = check_hazard_datasets(lat, lon)
    
    all_undesirables = google_undes.union(hazard_undes)
    # The QAP says 2 points deducted for each distinct undesirable. 
    return len(all_undesirables) * 2

In [ ]:
def is_food_desert(lat, lon):
    """
    The QAP references USDA Food Access data. 
    We check if this site’s census tract is flagged as a 'food desert.'
    That typically means the LILATracts_1And20 or similar = 1.
    
    We'll demonstrate using a shapefile or CSV that has polygons for each tract
    plus columns for the LILA flags.
    """
    try:
        fd_gdf = gpd.read_file(FOOD_DESERT_SHP)
        site_point = Point(lon, lat)
        for idx, row in fd_gdf.iterrows():
            if row.geometry.contains(site_point):
                # check if row says it's a food desert
                # e.g., row["LILATracts_1And20"] == 1
                # or row["LILATracts_halfAnd10"] == 1, etc. 
                # depends on QAP’s requirement
                if row.get("LILATracts_1And20", 0) == 1:
                    return True
        return False
    except:
        return False


In [ ]:
def check_grocery_store_points(lat, lon, pool_type):
    """
    Did the project get any grocery store points? 
    We'll replicate the grocery store portion from the Desirable scoring 
    to see if it qualifies for points. If not, then we can apply the food desert deduction.
    """
    # We'll specifically re-run just for 'grocery_store'
    config = DESIRABLE_AMENITIES["grocery_store"]
    dist = fetch_nearest_desirable(lat, lon, "grocery_store", config, pool_type)
    pts = compute_desirable_points(pool_type, config["group"], dist)
    return pts > 0

### Main Scoring Functions

In [41]:
def get_desirable_undesirable_score(lat, lon):
    start_time = time.time()

    """
    1) Determine pool
    2) Sum up desirable points (cap at 20)
    3) Subtract 2 points for each undesirable
    4) Check food desert => -2 if no grocery store points
    Return final score.
    """
    # 1) Which Pool?
    pool_type = determine_pool(lat, lon)
    
    # 2) Desirable
    desirable_pts = calculate_desirable_score(lat, lon, pool_type)
    
    # 3) Undesirable => each distinct category is -2
    # undesirable_deduction = compute_undesirable_deduction(lat, lon)
    
    # 4) Food desert => if site is in a food desert, and no grocery store points => -2
    # food_desert_penalty = 0
    # if is_food_desert(lat, lon):
    #     got_grocery_points = check_grocery_store_points(lat, lon, pool_type)
    #     if not got_grocery_points:
    #         food_desert_penalty = 2
    
    # final_score = desirable_pts - undesirable_deduction - food_desert_penalty
    print("get_desirable_undesirable_score took ", time.time() - start_time, "seconds")

    final_score = desirable_pts
    return final_score


In [55]:
site_lat = 30.98833
site_lon = -82.89667
score = identify_undesirable_activities(site_lat, site_lon, radius_miles=0.25)
print(score)


{'hazardous_inventory'}


In [42]:
if __name__ == "__main__":
    # Example usage: pick a lat/lon in GA
    site_lat = 30.818623
    site_lon = -83.265127
    score = get_desirable_undesirable_score(site_lat, site_lon)
    print(f"Final Desirable/Undesirable Score for site at ({site_lat}, {site_lon}): {score}")

google_places_search took  0.5584259033203125 seconds
get_route_distance_miles took  0.30393123626708984 seconds
get_route_distance_miles took  0.16967201232910156 seconds
get_route_distance_miles took  0.18464207649230957 seconds
fetch_nearest_desirable took  1.2173638343811035 seconds
google_places_search took  1.8515336513519287 seconds
get_route_distance_miles took  0.1295490264892578 seconds
get_route_distance_miles took  0.2137300968170166 seconds
get_route_distance_miles took  0.18445205688476562 seconds
get_route_distance_miles took  0.2518281936645508 seconds
get_route_distance_miles took  0.1691901683807373 seconds
get_route_distance_miles took  0.19678020477294922 seconds
get_route_distance_miles took  0.11493682861328125 seconds
get_route_distance_miles took  0.17223501205444336 seconds
get_route_distance_miles took  0.23444199562072754 seconds
get_route_distance_miles took  0.16070079803466797 seconds
get_route_distance_miles took  0.19941210746765137 seconds
get_route_dis

MAPPING SCORES

SCORES FOLLOW GRADIENT 

USE GRID BASED APPROACH 
- MAX AND MIN LAT OF Georgia, MIN AND MAX LONG OF GEORGIA 
- Generate random points between (pair wise points) 

USE LATTIS (OR SOMETHING) - PYTHON EQUIVALENT OF EXPAND.GRID in R


(can plug LAT, long, score dataframe into QGIS) 